In [43]:
import subprocess
import pandas as pd

cmd = f"find /opt/output/perm/path/*.json"
paths = subprocess.check_output(cmd, shell=True).decode().strip().split()
not_syscall_set = set()
# 대체
replace_dict = {"_exit":"231","capset":"126","ioctl":"16", "msgrcv":"70", "setns":"308","shutdown":"48","time":"201","uname":"63","__NR_vm86old":"436","__NR_socketcall":"437","stime":"308"}
for path in paths:
    EID = path.replace("/opt/output/perm/path/","").replace(".json","")
    with open(path, "r") as f:
        pathDict = json.load(f)
        for func, syspath_set in pathDict.items():
            for i, syspath in enumerate(syspath_set):
                for j, syscall in enumerate(syspath):
                    if not syscall.isdigit():
                        if pathDict[func][i][j] in replace_dict.keys():
                            pathDict[func][i][j] = replace_dict[syscall]
                        else:
                            not_syscall_set.add(syscall)
    with open(f"./new_path/{EID}.json", 'w', encoding='utf-8') as file:
        json.dump(pathDict, file)

In [44]:
cmd = f"find ./new_path/*.json"
paths = subprocess.check_output(cmd, shell=True).decode().strip().split()
# if "219)" in not_syscall_set:
new_pathDict = dict()
for path in paths:
    EID = path.replace("./new_path/","").replace(".json","")
    with open(path, "r") as f:
        pathDict = json.load(f)
        for func, syspath_set in pathDict.items():
            new_pathDict[func] = []
            for i, syspath in enumerate(syspath_set):
                new_syspath = syspath.copy()
                for syscall in syspath:
                    if syscall in not_syscall_set:
                        new_syspath.remove(syscall)
                new_pathDict[func].append(new_syspath)
    with open(f"./new_path/{EID}.json", 'w', encoding='utf-8') as file:
        json.dump(new_pathDict, file)
    new_pathDict = dict()